# Creating Expressions

In this notebook, we look at different options to create expressions. A strong focus will be set on the array-like operations: Since variables are represented in array-like structure, we benefit from a lot of well-knwon functionalities which we know from `numpy`, `pandas` or `xarray`.


These are for example

- `arithmetic` operations to create expressions
- `broadcasting` to combine smaller and larger arrays
- `.loc` to select a subset of the original array using indexes
- `.where` to select where the variable or expression should be active or not
- `.shift` to shift the whole array along one dimension
- `.groupby` to group by a key and apply operations on the groups 
- `.rolling` to perform a rolling operation and perform operations


.. hint::
   Nearly all of the functions and properties, that can be accessed from a `Variable`, can be accesses from a `LinearExpression` and `QuadraticExpression`.

Let's start by creating a model.

In [ ]:
import pandas as pd
import xarray as xr

import linopy

time = pd.Index(range(10), name="time")
port = pd.Index(list("abcd"), name="port")

m = linopy.Model()
x = m.add_variables(lower=0, coords=[time], name="x")
y = m.add_variables(lower=0, coords=[time, port], name="y")
m

## Arithmetic Operations

Arithmetic operations such as addition (`+`), subtraction (`-`), multiplication (`*`) can be used directly on the variables and expressions in Linopy. These operations are applied element-wise on the variables.

For example, if you want to create a new combined expr `z` that is the sum of `x` and `y`, you can do so as follows:

In [ ]:
z = x + y
z

.. note::
   In the addition, the variable `x` is broadcasted and the return value has the same set of dimensions as `y`.  

Similarly, you can subtract `y` from `x` or multiply `x` and `y` as follows:

In [ ]:
z = x - y
z

In [ ]:
z = x * y
z

In all cases, the returned shape is the same. Note that, the output type of the multiplication is a `QuadraticExpression` and not a `LinearExpression`.


The `z` expression, which carries along `x` and `y`, has different attributes such as `coord_dims`, `dims`, `size`.

In [ ]:
z.coord_dims

.. important::

	When combining variables or expression with dimensions of the same name and size, the first object will determine the coordinates of the resulting expression. For example:

In [ ]:
other_time = pd.Index(range(10, 20), name="time")
b = m.add_variables(coords=[other_time], name="b")
b

`b` has the same shape as `x`, but they have different coordinates. When we combine `x` and `b` the coordinates on dimension `time` will be taken from the first object and the coordinates of the subsequent object will be ignored:

In [ ]:
x + b

## Using `.loc` to select a subset

The `.loc` function allows you to select a subset of the array using indexes. This is useful when you want to apply operations to a specific subset of your variables.

For example, if you want to apply a summation to the variables `x` and `y` only for the first 5 time steps, you can do so as follows:

In [ ]:
x.loc[:5]

In [ ]:
x.loc[:5] + y.loc[:5]

which is the same as

In [ ]:
expr = x + y
expr.loc[:5]

In combination with the overwrite of the coordinates, this is useful when you need to combine different selections, like

In [ ]:
x.loc[:4] + y.loc[5:]

## Using `.where` to select active variables or expressions

The `.where` function allows you to select where the variable or expression should be active or not. This is useful when you want to apply constraints or operations only to a specific subset of your variables based on a condition. It is quite similar to the functionality of masking, that we showed earlier.

For example, if you want to create an sum of the variables `x` and `y` where `time` is greater than 2, you can do so as follows:

In [ ]:
mask = xr.DataArray(time > 2, coords=[time])
(x + y).where(mask)

We can use this to make a conditional summation:

In [ ]:
(x + y).where(mask) + xr.DataArray(5, coords=[time]).where(~mask, 0)

## Using `.shift` to shift the Variable along one dimension

The `.shift` function allows you to shift the whole array along one dimension. This is useful when you want to apply constraints or operations that involve a time delay or a shift in the time steps.

For example, if you want to apply a constraint that involves a one time step delay in the variables `x` and `y`, you can do so as follows:

In [ ]:
y - y.shift(time=1)

## Using `.groupby` to group by a key and apply operations on the groups

The `.groupby` function allows you to group by a key and apply operations on the groups. This is useful when you want to apply constraints or operations that involve a grouping of the time steps or any other dimension.

For example, if you want to apply a constraint that involves the sum of `x` and `y` over every two time steps, you can do so as follows:

In [ ]:
group_key = pd.Series(time.values // 2, index=time)
(x + y).groupby(group_key).sum()

## Using `.rolling` to perform a rolling operation

The `.rolling` function allows you to perform a rolling operation and apply operations. This is useful when you want to apply constraints or operations that involve a rolling window of the time steps or any other dimension.

For example, if you want to apply a constraint that involves the sum of `x` over a rolling window of 3 time steps, you can do so as follows:

In [ ]:
x.rolling(time=3).sum()